# Checkpoint surgery

In [1]:
import sys
import os

root_path = os.path.dirname(os.getcwd())
src_path = os.path.join(root_path, "src")
sys.path.insert(0, src_path)

In [2]:
import torch
new_ckpt = torch.load("ckpt-403.pt", map_location=torch.device('cpu'))

In [3]:
state_dict_old = torch.load("OLD_model_NEW_transform_2000EPOCH.pth")

In [12]:
#state_dict_new = new_ckpt["state_dict"]
#torch.save(state_dict_old,"state_dict_old.pt")
#torch.save(state_dict_new,"state_dict_new.pt")

In [5]:
old_keys = state_dict_old.keys()
new_keys = new_ckpt["state_dict"].keys()

In [6]:
key_intersection = set(old_keys) & set(new_keys)
print(key_intersection)

{'backbone.7.0.bn2.running_mean', 'backbone.4.0.conv2.weight', 'backbone.5.1.bn1.running_var', 'backbone.4.0.bn2.bias', 'backbone.4.0.bn1.running_var', 'backbone.6.0.bn1.num_batches_tracked', 'backbone.7.0.downsample.1.running_mean', 'backbone.6.1.bn2.num_batches_tracked', 'backbone.1.running_var', 'backbone.7.1.bn2.running_var', 'backbone.7.1.bn1.weight', 'backbone.5.0.downsample.1.bias', 'backbone.5.0.bn2.running_var', 'backbone.4.0.bn1.num_batches_tracked', 'backbone.5.0.bn2.num_batches_tracked', 'backbone.4.1.bn1.weight', 'backbone.7.1.conv2.weight', 'backbone.7.0.bn2.running_var', 'backbone.5.0.bn1.bias', 'backbone.6.1.bn1.num_batches_tracked', 'backbone.7.0.bn2.weight', 'backbone.7.0.bn1.bias', 'backbone.7.1.bn1.bias', 'backbone.5.1.conv2.weight', 'backbone.6.1.bn1.bias', 'backbone.4.0.bn2.num_batches_tracked', 'backbone.5.0.conv1.weight', 'backbone.6.0.bn2.num_batches_tracked', 'backbone.6.0.downsample.1.running_mean', 'backbone.7.1.bn1.num_batches_tracked', 'backbone.5.0.bn1.ru

In [7]:
key_not_matching_old = set(old_keys) - key_intersection
key_not_matching_new = set(new_keys) - key_intersection

print(key_not_matching_old)
print(key_not_matching_new)

{'projection_head.0.bias', 'projection_head.2.bias', 'projection_head.0.weight', 'projection_head.2.weight'}
{'projection.head.0.bias', 'projection.head.2.weight', 'projection.head.0.weight', 'projection.head.2.bias'}


In [8]:
import copy

conversion_new2old = {
    'projection.head.0.weight': 'projection_head.0.weight',
    'projection.head.2.weight': 'projection_head.2.weight',
    'projection.head.0.bias': 'projection_head.0.bias',
    'projection.head.2.bias': 'projection_head.2.bias',
}

surgery_ckpt = copy.deepcopy(new_ckpt)
surgery_ckpt['neptune_experiment_id'] = 'TIS-405'

In [10]:
for key,v in surgery_ckpt["state_dict"].items():
    try:
        surgery_ckpt["state_dict"][key] = state_dict_old[key]
    except KeyError:
        surgery_ckpt["state_dict"][key] = state_dict_old[conversion_new2old[key]]


In [11]:
torch.save(surgery_ckpt,"ckpt-TIS-405.pt")